<a href="https://colab.research.google.com/github/dcandrader/IA_2025.2/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Carga de librerias y recursos
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from progressbar import progressbar as pbar

In [ ]:
#Carga de datos directamente de Kaggle
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.65GB/s]


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
#Lectura de datos
d_train = pd.read_csv('train.csv')

orden_valor = [
    'bajo',
    'medio-bajo',
    'medio-alto',
    'alto'
  ]

d_train['RENDIMIENTO_GLOBAL'] = pd.Categorical(
    d_train['RENDIMIENTO_GLOBAL'],
    categories=orden_valor,
    ordered=True
  ).codes

d_test = pd.read_csv('test.csv')
lentr = len(d_train)
d_train

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,2,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,0,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,0,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,3,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,1,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,2,0.237,0.271,0.271,0.311
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,...,N,No,Si,Si,Secundaria (Bachillerato) incompleta,0,0.314,0.240,0.278,0.260
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,1,0.286,0.240,0.314,0.287
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,...,N,Si,Si,No,Primaria completa,0,0.132,0.426,0.261,0.328


In [ ]:
#Función para preprocesado de datos

def procesado_datos(d):
  for col in d.select_dtypes(include='object').columns:
    d[col] = d[col].astype('category')

  orden = sorted(d['PERIODO_ACADEMICO'].unique())
  mapa_periodos = {valor: i for i, valor in enumerate(orden)}

  d['PERIODO_ACADEMICO'] = d['PERIODO_ACADEMICO'].map(mapa_periodos)
  d['PERIODO_ACADEMICO'] = d['PERIODO_ACADEMICO'].astype('Int64')

  cols_normal = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
  scaler = StandardScaler()
  d_train[cols_normal] = scaler.fit_transform(d_train[cols_normal])

  orden_valor = [
    'Menos de 500 mil',
    'Entre 500 mil y menos de 1 millón',
    'Entre 1 millón y menos de 2.5 millones',
    'Entre 2.5 millones y menos de 4 millones',
    'Entre 4 millones y menos de 5.5 millones',
    'Entre 5.5 millones y menos de 7 millones',
    'Más de 7 millones',
    'No pagó matrícula',
    'Desconocido'
  ]
  d['E_VALORMATRICULAUNIVERSIDAD'] = pd.Categorical(
    d['E_VALORMATRICULAUNIVERSIDAD'],
    categories=orden_valor,
    ordered=True
  ).codes

  orden_horas = [
    '0',
    'Menos de 10 horas',
    'Entre 11 y 20 horas',
    'Entre 21 y 30 horas',
    'Más de 30 horas',
    'Desconocido'
  ]
  d['E_HORASSEMANATRABAJA'] = pd.Categorical(
    d['E_HORASSEMANATRABAJA'],
    categories=orden_horas,
    ordered=True
  ).codes

  orden_estrato = [
    'Estrato 1',
    'Estrato 2',
    'Estrato 3',
    'Estrato 4',
    'Estrato 5',
    'Estrato 6',
    'Sin Estrato',
    'Desconocido'
  ]

  d['F_ESTRATOVIVIENDA'] = pd.Categorical(
    d['F_ESTRATOVIVIENDA'],
    categories=orden_estrato,
    ordered=True
  ).codes

  orden_educacion = [
    'Ninguno',
    'Primaria incompleta',
    'Primaria completa',
    'Secundaria (Bachillerato) incompleta',
    'Secundaria (Bachillerato) completa',
    'Técnica o tecnológica incompleta',
    'Técnica o tecnológica completa',
    'Educación profesional incompleta',
    'Educación profesional completa',
    'Postgrado',
    'No Aplica',
    'No sabe',
    'Desconocido'
  ]

  for col in ['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']:
    d[col] = pd.Categorical(
        d[col],
        categories=orden_educacion,
        ordered=True
    ).codes

  return d

In [ ]:
#Preparación para entrenamiento
d_train = d_train.dropna()
d_procesados = procesado_datos(d_train)
d_columnas = [i for i in d_procesados.columns if i!="RENDIMIENTO_GLOBAL"]
Xtr = d_procesados[d_columnas]
ytr = d_procesados["RENDIMIENTO_GLOBAL"]
Xts = procesado_datos(d_test)

/tmp/ipython-input-729217109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[col] = d[col].astype('category')
/tmp/ipython-input-729217109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[col] = d[col].astype('category')
/tmp/ipython-input-729217109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [ ]:
print (Xtr.shape, ytr.shape)
print (Xts.shape)

(628896, 20) (628896,)
(296786, 20)


In [ ]:
Xtr

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,5,ENFERMERIA,BOGOTÁ,5,1,2,Si,5,Si,Si,N,No,Si,Si,9,0.368091,-0.795004,1.324417,-0.169550
1,645256,5,DERECHO,ATLANTICO,3,0,2,No,6,Si,No,N,No,Si,No,5,0.269929,-0.706702,0.665910,-0.237725
2,308367,4,MERCADEO Y PUBLICIDAD,BOGOTÁ,3,4,2,Si,4,Si,No,N,No,No,Si,4,0.144995,-0.719317,1.141499,-0.237725
3,470353,2,ADMINISTRACION DE EMPRESAS,SANTANDER,4,0,3,Si,11,Si,No,N,No,Si,Si,4,1.822680,-1.249129,-0.797440,-1.919364
4,989032,5,PSICOLOGIA,ANTIOQUIA,3,3,2,Si,2,Si,Si,N,No,Si,Si,2,0.314548,-0.492255,0.409824,0.444021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,2,BIOLOGIA,LA GUAJIRA,1,2,1,Si,4,Si,No,N,Si,Si,Si,3,-0.390437,-0.000286,-0.102349,0.830343
692496,754213,5,PSICOLOGIA,NORTE SANTANDER,3,4,2,Si,1,Si,No,N,No,Si,Si,3,0.296700,-0.391338,0.153737,-0.328624
692497,504185,0,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,2,1,2,Si,4,Si,No,N,Si,Si,Si,3,0.046832,-0.391338,1.470752,0.284947
692498,986620,2,PSICOLOGIA,TOLIMA,3,1,0,No,2,No,No,N,Si,Si,No,2,-1.327442,1.954973,-0.468186,1.216666


In [ ]:
!pip install catboost
from catboost import CatBoostClassifier

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00


In [ ]:
cat_cols = Xtr.select_dtypes(include=['object', 'category']).columns
cat_cols = list(cat_cols)
cat_features = [Xtr.columns.get_loc(c) for c in cat_cols]

from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=500,
    depth=6,
    learning_rate=0.1,
    loss_function="MultiClass",
    task_type="GPU",
    verbose=100
)

model.fit(Xtr, ytr, cat_features=cat_features)

0:	learn: 1.3682961	total: 30.8ms	remaining: 15.4s
100:	learn: 1.2049350	total: 2s	remaining: 7.88s
200:	learn: 1.1965222	total: 3.89s	remaining: 5.79s
300:	learn: 1.1918900	total: 5.81s	remaining: 3.84s
400:	learn: 1.1887259	total: 7.64s	remaining: 1.89s
499:	learn: 1.1861226	total: 10.9s	remaining: 0us


In [ ]:
# Reemplazar NaN en categóricas por string
for col in Xts.select_dtypes(include=["category", "object"]).columns:
    Xts[col] = Xts[col].astype("category")
    Xts[col] = Xts[col].cat.add_categories(["Desconocido"])
    Xts[col] = Xts[col].fillna("Desconocido")

In [ ]:
preds_ts = model.predict(Xts)
preds_ts

array([[0],
       [1],
       [3],
       ...,
       [2],
       [3],
       [3]])

In [ ]:
preds_ts = preds_ts.ravel()
preds_ts

array([0, 1, 3, ..., 2, 3, 3])

In [ ]:
submission = pd.DataFrame({
    "ID": d_test["ID"],
    "RENDIMIENTO_GLOBAL": preds_ts
})
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,0
1,98545,1
2,499179,3
3,782980,0
4,785185,0


In [ ]:
submission["RENDIMIENTO_GLOBAL"] = submission["RENDIMIENTO_GLOBAL"].map({
    0: "bajo",
    1: "medio-bajo",
    2: "medio-alto",
    3: "alto",
}).astype("category")

submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-bajo
2,499179,alto
3,782980,bajo
4,785185,bajo


In [ ]:
submission.to_csv("Andrade5.csv", index=False)

In [ ]:
from google.colab import files
files.download("Andrade5.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>